In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"America/Mexico_City\"")

In [ ]:
hex_project_id = _hex_json.loads("\"c46b0969-e1f4-480f-9dc4-74eca3d2a92f\"")

In [ ]:
hex_project_name = _hex_json.loads("\"2_euro2024_data_transformation\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

## Data Transformation
---

**Time: 15 mins**

- Leverage historical results and rankings
- Normalize the data to extract meaningful features
- Determine our target variable


In [ ]:
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import Window
from snowflake.snowpark import functions as F   
from snowflake.snowpark.functions import udf, udtf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, StructField, StructType, DateType
    
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import hextoolkit
hex_snowflake_conn = hextoolkit.get_data_connection('SCS-SIMON-EURO2024')
session = hex_snowflake_conn.get_snowpark_session()

In [ ]:
# add version tracking
app_tag = {
    "origin": "sf_sit",
    "name": "hol_sport_predict",
    "version": '{major: 1, minor: 0}'
}

session.query_tag = app_tag

In [ ]:
user_name = session.sql('select current_user()').collect()[0][0]

### Data Transformation (Part 1)


Get the home and away team ranks, at the time of the match date - we will use this to calculate the difference between ranks

In [ ]:
# Create a DataFrame by filtering and sorting match history from a database table
# Start by accessing the 'results' table from the session
# 
# Filter the data to include only records with a date on or after January 1, 1994,
# and exclude records where the 'tournament' column is 'Friendly'
#
# Finally, sort the resulting DataFrame by the 'id' column

df_match_history = (
    session.table("results")
    .filter(
        (F.col("date") >= "1994-01-01") &
        (F.col("tournament") != "Friendly")
    )
    .sort(F.col("id"))
)

# Create another DataFrame by sorting the 'rankings' table from the session
# Sort the data by the 'rank_date' column in ascending order

df_rank = session.table("rankings").sort(F.col("rank_date"), ascending=True)

In [ ]:
df_match_history.show(10)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"DATE"      |"HOME_TEAM"           |"AWAY_TEAM"    |"HOME_TEAM_SCORE"  |"AWAY_TEAM_SCORE"  |"TOURNAMENT"                                |"CITY"      |"COUNTRY"             |"NEUTRAL"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|19168  |1994-01-19  |Suriname              |French Guiana  |1                  |0                  |CFU Caribbean Cup qualification             |Paramaribo  |Suriname              |0          |
|19171  |1994-01-21  |Guyana                |French Guiana  |1                  |1                  |CFU Caribbean Cup qualification             |Paramaribo  |Suriname              |1          |
|19173  |1994-01-23  |Bar

In [ ]:
# we join the history with rank - this will explode the table so we'll have all fixtures and all ranks 
# we'll want to get the fixture record with the most recent rank date relevant to the match date
df_match_rank_home = df_match_history.join(
    df_rank,
    ((df_rank['rank_date'] <= df_match_history['date']) &
     (df_match_history['home_team'] == df_rank['country_full'])),
    'left'
).sort(['id', df_rank['rank_date'].desc()])

window_spec = Window.partition_by('id').order_by(df_rank['rank_date'].desc())
df_match_rank_home = df_match_rank_home.select(
    F.row_number().over(window_spec).alias('row_number'),
    'id',
    'rank',
    'rank_date'
).filter(F.col('row_number') == 1)

df_match_rank_home = df_match_rank_home.withColumnRenamed('id', 'home_id') \
.withColumnRenamed('rank', 'home_team_rank') \
.drop('row_number', 'rank_date')

# and next we'll want to do the same thing for the away team as well...
df_match_rank_away = df_match_history.join(
    df_rank,
    (
        (df_rank["rank_date"] <= df_match_history["date"])
        & (df_match_history["away_team"] == df_rank["country_full"])
    ),
    "left",
).sort(["id", F.desc("rank_date")])

df_match_rank_away = (
    df_match_rank_away.select(
        F.row_number()
        .over(Window.partitionBy("id").orderBy(F.desc("rank_date")))
        .alias("row_number"),
        "id",
        "rank",
        "rank_date",
    )
    .filter(F.col("row_number") == 1)
    .drop("row_number", "rank_date")
)

# Rename columns to reflect they are for the away team
df_match_rank_away = df_match_rank_away.withColumnRenamed(
    "id", "away_id"
).withColumnRenamed("rank", "away_team_rank")

# now we join these 2 dataframes together
df_match_rank = (
    df_match_rank_home.join(
        df_match_rank_away,
        df_match_rank_home["home_id"] == df_match_rank_away["away_id"],
        "left",
    )
    .select(
        df_match_rank_home["home_id"].alias("id"),
        df_match_rank_home["home_team_rank"],
        df_match_rank_away["away_team_rank"]
    )
    .sort("id")
)

# get rid of the rows with null values
df_match_rank = df_match_rank.filter(
    df_match_rank.col('home_team_rank').is_not_null() & df_match_rank.col('away_team_rank').is_not_null()
)

df_match_rank = df_match_rank.na.drop()

df_match_rank.show(10)

-----------------------------------------------
|"ID"   |"HOME_TEAM_RANK"  |"AWAY_TEAM_RANK"  |
-----------------------------------------------
|19173  |114.0             |105.0             |
|19176  |117.0             |136.0             |
|19178  |143.0             |105.0             |
|19182  |114.0             |143.0             |
|19192  |26.0              |30.0              |
|19194  |51.0              |150.0             |
|19195  |26.0              |150.0             |
|19197  |51.0              |30.0              |
|19199  |30.0              |150.0             |
|19201  |51.0              |26.0              |
-----------------------------------------------



### Data Transformation(Part 2)

- Get the recent performance of the home and away team
- For each match, we are going to fetch the trailing 10 games (for both home and away teams) and calculate recent performance - the idea here is, does recent performance have any impact on the result of a game?  


In [ ]:
# UDTF to calculate the trailing performance of a team, with a given set of games
# It will return:
# - no. of wins
# - no. of losses
# - goal difference (goals scored - goals conceded)

output_struct = StructType([
    StructField("ttl_wins", FloatType()),
    StructField("ttl_losses", FloatType()),
    StructField("ttl_draws", FloatType()),
    StructField("goal_diff",FloatType())
])

@udtf(output_schema = output_struct,
    input_types = [IntegerType(),DateType(), FloatType(), FloatType(), IntegerType()],
    name = "calc_performance", 
    session = session,
    is_permanent=True, 
    stage_location="@python_load",
    packages=["pandas"], 
    replace=True)

class calc_ttl_performance:
    def __init__(self):
        self.fixture_id = []
        self.date = []
        self.goals_for = []
        self.goals_against = []
        self.is_home = []

    def process(self, fixture_id, date, goals_for, goals_against, is_home):
        self.fixture_id.append(fixture_id)
        self.date.append(date)
        self.goals_for.append(goals_for)
        self.goals_against.append(goals_against)
        self.is_home.append(is_home)
    
    def end_partition(self):
        df = pd.DataFrame(zip(self.fixture_id, self.date, self.goals_for, self.goals_against, self.is_home), 
                            columns=['fixture_id', 'date', 'home_team_goals', 'away_team_goals', 'is_home'])

        df['goals_scored'] = np.where(df['is_home'] == 1, df['home_team_goals'], df['away_team_goals'])  
        
        df['goals_conceded'] = np.where(df['is_home'] == 1, df['away_team_goals'], df['home_team_goals']) 

        df['wins'] = np.where((df['home_team_goals'] > df['away_team_goals']) & (df['is_home'] == 1), 1, 
                     np.where((df['home_team_goals'] < df['away_team_goals']) & (df['is_home'] != 1), 1, 0))  
        
        df['draws'] = np.where(df['home_team_goals'] == df['away_team_goals'], 1, 0)  

        df['losses'] = np.where((df['home_team_goals'] < df['away_team_goals']) & (df['is_home'] == 1), 1, 
                       np.where((df['home_team_goals'] > df['away_team_goals']) & (df['is_home'] != 1), 1, 0))  


        ttl_wins = np.sum(df['wins'])
        ttl_losses = np.sum(df['losses'])
        ttl_draws = np.sum(df['draws'])
        goal_diff = np.sum(df['goals_scored']) - np.sum(df['goals_conceded'])

        if np.isnan(goal_diff):
            goal_diff = 0

        yield (ttl_wins, ttl_losses, ttl_draws, goal_diff)

In [ ]:
df_match_history = (
    session.table("results")
    .filter(
        (F.col("date") >= "1994-01-01") &
        (F.col("tournament") != "Friendly")
    )
    .sort(F.col("id"), ascending=True)
)

df_history = (
    session.table("results")
    .sort(F.col("id"), ascending=True)
)

In [ ]:
# Starting with the home team
# For each match we want to get all the prior matches
# Then in the next step we will limit that to the prior 5

df_home_team = (
    df_match_history.join(
        df_history,
        (
            (
                (df_match_history["home_team"] == df_history["home_team"]) | (df_match_history["home_team"] == df_history["away_team"])
            )
            & (df_history["date"] < df_match_history["date"])
        ),
        "left",
    )
    .select(
        df_match_history["id"].alias("id"),
        df_match_history["date"].alias("match_date"),
        df_match_history["home_team"].alias("team_1"),
        df_match_history["away_team"].alias("team_2"),
        df_match_history["neutral"].alias("neutral"),
        df_history["home_team"].alias("home_team"),
        df_history["away_team"].alias("away_team"),
        df_history["date"].cast(DateType()).alias("history_date"),
        df_history["home_team_score"].alias("home_team_score"),
        df_history["away_team_score"].alias("away_team_score"),
    )
    .with_column(
        "is_home",
        F.when(df_match_history["home_team"] == df_history["home_team"], 1).otherwise(0),
    )
    .with_column(
        "row_number",
        F.row_number().over(Window.partition_by("id").order_by(F.col("history_date").desc())),
    )
    .filter(F.col("row_number") <= 15)
    .drop("row_number")
)

# we use our UDTF to pass through the games partitioned by fixture id - there should be 5 games per partition

perf_udtf = F.table_function("calc_performance")

df_home_team = df_home_team.join_table_function(
    perf_udtf(
        df_home_team.col('id').cast(IntegerType()),
        df_home_team.col('history_date').cast(DateType()),
        df_home_team.col('home_team_score').cast(FloatType()),
        df_home_team.col('away_team_score').cast(FloatType()),
        df_home_team.col('is_home').cast(IntegerType())).over(partition_by='id',order_by=['history_date'])
    )\
    .select(
        F.col('id').alias('home_fixture_id'),
        F.col('goal_diff').alias('home_goal_diff'),
        F.col('ttl_wins').alias('home_ttl_wins'),
        F.col('ttl_losses').alias('home_ttl_losses')
    )

df_home_team.show(10)

------------------------------------------------------------------------------
|"HOME_FIXTURE_ID"  |"HOME_GOAL_DIFF"  |"HOME_TTL_WINS"  |"HOME_TTL_LOSSES"  |
------------------------------------------------------------------------------
|30132              |8.0               |2.0              |3.0                |
|29096              |12.0              |7.0              |3.0                |
|40577              |-19.0             |3.0              |9.0                |
|36513              |4.0               |8.0              |6.0                |
|21715              |10.0              |6.0              |4.0                |
|44789              |5.0               |5.0              |4.0                |
|37421              |38.0              |14.0             |1.0                |
|20022              |-6.0              |5.0              |7.0                |
|43842              |-4.0              |4.0              |6.0                |
|22487              |7.0               |7.0         

In [ ]:
# Now the same for the away team
# Again, we use our UDTF to pass through the games partitioned by fixture id - there should be 5 games per partition
df_away_team = (
    df_match_history.join(
        df_history,
        (
            (
                (df_match_history["away_team"] == df_history["home_team"])
                | (df_match_history["away_team"] == df_history["away_team"])
            )
            & (df_history["date"] < df_match_history["date"])
        ),
        "left",
    )
    .select(
        df_match_history["id"].alias("id"),
        df_match_history["date"].alias("match_date"),
        df_match_history["home_team"].alias("team_1"),
        df_match_history["away_team"].alias("team_2"),
        df_match_history["neutral"].alias("neutral"),
        df_history["home_team"].alias("home_team"),
        df_history["away_team"].alias("away_team"),
        df_history["date"].cast(DateType()).alias("history_date"),
        df_history["home_team_score"].alias("home_team_score"),
        df_history["away_team_score"].alias("away_team_score"),
    )
    .with_column(
        "is_home",
        F.when(df_match_history["away_team"] == df_history["home_team"], 1).otherwise(
            0
        ),
    )
    .with_column(
        "row_number",
        F.row_number().over(Window.partition_by("id").order_by(F.col("history_date").desc())),
    )
    .filter(F.col("row_number") <= 15)
    .drop("row_number")
)

perf_udtf = F.table_function("calc_performance")

df_away_team = df_away_team.join_table_function(
    perf_udtf(
        df_away_team.col('id').cast(IntegerType()),
        df_away_team.col('history_date').cast(DateType()),
        df_away_team.col('home_team_score').cast(FloatType()),
        df_away_team.col('away_team_score').cast(FloatType()),
        df_away_team.col('is_home').cast(IntegerType())).over(partition_by='id',order_by=['history_date'])
    ) \
    .select(
        F.col('id').alias('away_fixture_id'),
        F.col('goal_diff').alias('away_goal_diff'),
        F.col('ttl_wins').alias('away_ttl_wins'),
        F.col('ttl_losses').alias('away_ttl_losses')    
    )

df_away_team.show(10)

------------------------------------------------------------------------------
|"AWAY_FIXTURE_ID"  |"AWAY_GOAL_DIFF"  |"AWAY_TTL_WINS"  |"AWAY_TTL_LOSSES"  |
------------------------------------------------------------------------------
|22868              |11.0              |8.0              |5.0                |
|28688              |15.0              |10.0             |5.0                |
|42188              |2.0               |6.0              |8.0                |
|30165              |4.0               |5.0              |3.0                |
|34694              |-14.0             |3.0              |9.0                |
|31964              |12.0              |9.0              |4.0                |
|46538              |3.0               |5.0              |7.0                |
|26501              |11.0              |7.0              |5.0                |
|29001              |-9.0              |3.0              |7.0                |
|39975              |3.0               |6.0         

In [ ]:
# now we join the home and away performance into its own dataframe - these can be joined later on and used as features.
df_team_perf = df_home_team \
    .join(
        df_away_team,
        (
            df_away_team.col('away_fixture_id') == df_home_team.col('home_fixture_id')
        ),
        'left'
    ).drop(
        'away_fixture_id'
    ).rename(
        F.col("home_fixture_id"), 'id'
    )

df_team_perf.show(10)

---------------------------------------------------------------------------------------------------------------------------
|"ID"   |"HOME_GOAL_DIFF"  |"HOME_TTL_WINS"  |"HOME_TTL_LOSSES"  |"AWAY_GOAL_DIFF"  |"AWAY_TTL_WINS"  |"AWAY_TTL_LOSSES"  |
---------------------------------------------------------------------------------------------------------------------------
|25470  |-3.0              |4.0              |5.0                |4.0               |4.0              |5.0                |
|41119  |0.0               |9.0              |5.0                |-5.0              |3.0              |9.0                |
|23213  |6.0               |6.0              |3.0                |-16.0             |4.0              |8.0                |
|40027  |-7.0              |4.0              |5.0                |5.0               |7.0              |5.0                |
|23709  |5.0               |6.0              |3.0                |10.0              |9.0              |2.0                |
|30849  

### Data Transformation (Part 3)
- Combine all features in a single dataframe
- Save the dataframe in a FINAL_DATA table

Initial tests found low accuracy (~55%) for multi-class predictions (i.e prediction Home Win, Draw, Away Win) so we're trying a different approach:

- Reduce the outcome of the match to Home Win binary outcome (alternative is Draw and Away Win)
- Run the prediction model twice, first for the Home Team and then for the Away Team
- Combine the outcome of this like so:


| Team 1 Outcome   | Team 2 Outcome   | Implied Result
| ----------- | ----------- |--------
| 1           | 0           |  Team 1 Win
| 0           | 1           | Team 2 Win
| 1           | 1           | Draw
| 0           | 0           | Draw


Therefore, our label will be 1 or 0 for model training.


In [ ]:
# UDF to bin the outcome into:
# home win = 1
# away win or draw = 0

@udf(name='convert_score',  
     is_permanent=True,
     replace=True,
     session=session,
     stage_location='python_load')

def convert_score(x:int, y:int) -> int: 
    if x > y : # home win
        return 1

    return 0

In [ ]:
df_match_history_full = session.table('results') \
    .filter(
        (F.col("date") >= "1994-01-01") &
        (F.col("tournament") != "Friendly")
    ).sort(F.col('id'), ascending=False
)

In [ ]:
df_match_history_full = session.table('results') \
    .filter(
        (F.col("date") >= "1994-01-01") &
        (F.col("tournament") != "Friendly")
    ).sort(F.col('id'), ascending=False
)

df_match_history_full = df_match_history_full.join(
    df_match_rank,
    (
        df_match_history_full.col('id') == df_match_rank.col('id')
    ),
    'left'
).join(
    df_team_perf,
    (
        df_match_history_full.col('id') == df_team_perf.col('id')
    ),
    'left'
).select(
    df_match_history_full.col('id').alias('id'),
    df_match_history_full.col('neutral'),
    df_team_perf.col('home_goal_diff').alias('team_1_goal_diff'),
    df_team_perf.col('home_ttl_wins').alias('team_1_ttl_wins'),
    df_team_perf.col('home_ttl_losses').alias('team_1_ttl_losses'),
    df_team_perf.col('away_goal_diff').alias('team_2_goal_diff'),
    df_team_perf.col('away_ttl_wins').alias('team_2_ttl_wins'),
    df_team_perf.col('away_ttl_losses').alias('team_2_ttl_losses'),
    (df_match_rank.col('home_team_rank') - df_match_rank.col('away_team_rank')).cast(IntegerType()).alias('team_1_vs_team_2_rank'),
    F.call_function(
        'convert_score',
        df_match_history_full.col('home_team_score').cast(IntegerType()),
        df_match_history_full.col('away_team_score').cast(IntegerType())
        ).alias('game_outcome')
).sort(
    F.col('id'), ascending=True
).na.drop()

# save a specific version for this user
df_match_history_full.write.save_as_table(f'final_data_{user_name}', mode='overwrite')

session.table(f'final_data_{user_name}').show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"NEUTRAL"  |"TEAM_1_GOAL_DIFF"  |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_1_VS_TEAM_2_RANK"  |"GAME_OUTCOME"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|43686  |0          |38.0                |12.0               |0.0                  |2.0                 |8.0                |3.0                  |-10                      |1               |
|43688  |1          |-2.0                |5.0                |3.0                  |-13.0               |3.0                |9.0                  |-12                      |0               |
|43689  |1          |12.0                |9.0

Our final dataset contains the following features:

| Feature   | Description   |
| ----------- | ----------- |
| NEUTRAL           | Specifies whether the game is played in Team 1's home location. 0 = No, 1 = Yes. In theory, home advantage should have an effect  |
| TEAM_1_GOAL_DIFF| Last 10 goal difference of Team 1. Positive means they scored more than they conceded.
| TEAM_1_TTL_WINS | How many games Team 1 have won in the trailing 10 games (relative to the date of the current game)
| TEAM_1_TTL_LOSSES | How many games Team 1 have lost in the trailing 10 games (relative to the date of the current game)
| TEAM_2_GOAL_DIFF| Last 10 goal difference of Team 2. Positive means they scored more than they conceded.
| TEAM_2_TTL_WINS | How many games Team 2 have won in the trailing 10 games (relative to the date of the current game)
| TEAM_2_TTL_LOSSES | How many games Team 2 have lost in the trailing 10 games (relative to the date of the current game)
| TEAM_1_VS_TEAM_2_RANK| The difference in rank between Team 1 and Team 2 | 
